In [2]:
import networkx as nx
import random
from collections import defaultdict

def l_truncated_random_walk(graph, start, target, max_length):
    """Generate an l-truncated random walk from 'start' until it reaches 'target' or 'max_length',
    without revisiting nodes in the same walk."""
    path = [start]
    current = start
    visited = set([start])  # Track visited nodes to avoid revisiting

    for _ in range(max_length - 1):  # Walk up to max_length steps
        if current == target:  # Stop if the target node is reached
            break
        neighbors = [n for n in graph.neighbors(current) if n not in visited]  # Avoid revisiting nodes
        if not neighbors:  # If no unvisited neighbors are left, stop the walk
            break
        current = random.choice(neighbors)  # Randomly pick the next unvisited node
        path.append(current)
        visited.add(current)  # Mark the node as visited

    return path

def INITIALIZATION(graph, v, Q, l, epsilon):
    """Implementation of Algorithm 2 using l-truncated random walks with NetworkX."""
    n = len(graph)
    rho = int((n * (epsilon ** -2)) * (1 / l))  # Estimate of rho
    C = defaultdict(float)  # Store contributions per node
    R = {}  # Effective resistance array
    T = {v}  # Target node set containing only 'v'
    W = []  # Collection of walks

    # Step 2: Generate walks for each edge multiple times
    for (i, j) in graph.edges():
        if v not in(i, j):
            for _ in range(rho):
                # Generate two l-truncated random walks from the edge endpoints
                w1 = l_truncated_random_walk(graph, i, v, l)  # Walk from i
                w2 = l_truncated_random_walk(graph, j, v, l)  # Walk from j
                # Only combine walks if both reach the target node v
                if w1[-1] == v and w2[-1] == v:
                    # Combine walks (reverse the second to connect properly)
                    
                    combined_walk = w1 + w2[::-1][1:]
                    W.append(combined_walk)

    # Step 3: Process each walk and update contributions
    for walk in W:
        visited = set()  # Track nodes encountered for the first time
        for p, u in enumerate(walk):
            if u not in visited and u in Q:  # Only update if u is in Q
                visited.add(u)
                # Compute distance from u to the closest endpoint (either start or v)
                length = len(walk)
                # Update contribution for node u
                C[u] += 1 / (rho * length + 1)

    # Step 4: Compute effective resistance array R
    for u in C:
        R[u] = 1 / C[u]

    return R, W
# Create your graph using NetworkX
G = nx.Graph()
nodes = [0, 1, 2, 3, 4]
edges = [(0, 2), (4, 2), (1, 2), (1, 3), (2, 3), (1, 4)]
G.add_nodes_from(nodes)
G.add_edges_from(edges)

v = 0  # Target node
Q = {1, 2, 3, 4}  # Subset of nodes of interest
l = 5  # Maximum walk length
epsilon = 0.005  # Error parameter

R, W = INITIALIZATION(G, v, Q, l, epsilon)
print("Effective Resistance Array:", R)

Effective Resistance Array: {1: 7.851500417932217, 3: 9.675127711454744, 2: 6.752408225253967, 4: 9.660368063772891}


In [12]:
import networkx as nx
from collections import defaultdict

def DELETEEDGE(G:nx.Graph, W, v, Q, P):
    """Implements Algorithm 3: DELETEEDGE for computing edge removal and updating effective resistances."""
    n = G.number_of_nodes
    # Step 1: Create a node-walk map to identify walks where nodes appear
    node_walk_map = defaultdict(list)  # Maps nodes to the walks they appear in
    for walk in W:
        for node in walk:
            node_walk_map[node].append(tuple(walk))  # Use tuple to store the walk

    # Step 2: Initialize result dictionary for edge resistances
    R_v = {}  # Store effective resistance values for edges

    # Step 3: Iterate over all edges in the graph
    for (x, y) in G.edges():
        # Get the list of walks where either x or y appears
        W_tilde = node_walk_map[x] + node_walk_map[y]  # Concatenate lists of walks

        # Initialize edge weight updates for H_u
        edge_weights = defaultdict(float)  # Maps nodes u to updated weights

        # Process each walk in W_tilde
        for walk in W_tilde:
            for u in set(walk) & Q:  # Only consider u in both walk and Q
                T = {u, v, x, y}  # Node set including u, v, x, and y
                # Update the weight for u (increment by 1 for simplicity)
                edge_weights[u] += 1

        # Step 5: Check if removing the edge disconnects the graph
        G_temp = G.copy()
        G_temp.remove_edge(x, y)
        if not nx.is_connected(G_temp):
            R_v[(x, y)] = 0  # Set effective resistance to 0 if disconnected
        else:
            # If still connected, compute the effective resistance
            R_v[(x, y)] = sum(1 / edge_weights[u] for u in Q if edge_weights[u] > 0)

    return R_v # Return the resistance values

# Example usage of the delete_edge function
G = nx.Graph()
nodes = [0, 1, 2, 3, 4]
edges = [(0, 2), (4, 2), (1, 2), (1, 3), (2, 3), (1, 4)]
G.add_nodes_from(nodes)
G.add_edges_from(edges)

# Example random walks (W) collected

v = 2  # Target node
Q = {1, 3}  # Nodes of interest
P = set()  # Initially no removed edges

# Compute the effective resistances after processing edges
R_v = DELETEEDGE(G, W, v, Q, P)
print("Effective Resistances:", R_v)


Effective Resistances: {(0, 2): 0, (1, 2): 2.0850575774351657e-05, (1, 3): 2.855226043458345e-05, (1, 4): 3.102372342512523e-05, (2, 4): 2.4726676844559776e-05, (2, 3): 2.3137023082314948e-05}


In [4]:
# Example usage of the delete_edge function
G = nx.Graph()
nodes = [0, 1, 2, 3, 4]
edges = [(0, 2), (4, 2), (1, 2), (1, 3), (2, 3), (1, 4)]
G.add_nodes_from(nodes)
G.add_edges_from(edges)

# Example random walks (W) collected

v = 2  # Target node
Q = {1, 3}  # Nodes of interest
P = set()  # Initially no removed edges

# Compute the effective resistances after processing edges
R_v = DELETEEDGE(G, W, v, Q, P)
print("Effective Resistances:", R_v)

Effective Resistances: {(0, 2): 0, (1, 2): 2.0850575774351657e-05, (1, 3): 2.855226043458345e-05, (1, 4): 3.102372342512523e-05, (2, 4): 2.4726676844559776e-05, (2, 3): 2.3137023082314948e-05}


# Algorithm 4 approxisc

In [ ]:
def APPROXISC(G: nx.Graph, k, v, l, epsilon):
    P = {}
    for i in range(k):
        R, W = INITIALIZATION(G, v, Q, l, epsilon)
        R_v = DELETEEDGE(G, W, v, Q, P)
        arg_max = max(R_v)
        P[i] = arg_max
        G.remove_edges_from([arg_max])
    return P
# Example usage of the delete_edge function
G = nx.Graph()
nodes = [0, 1, 2, 3, 4]
edges = [(0, 2), (4, 2), (1, 2), (1, 3), (2, 3), (1, 4)]
G.add_nodes_from(nodes)
G.add_edges_from(edges)
k = 2
v = 2
l = 5
epsilon = 0.005
P = APPROXISC(G, k, v, l, epsilon)
print(f"Top-k edges need to remove {P}")

# FASTICM

In [78]:
import math
def FASTICM(G: nx.Graph, k, v, l, alpha, phi):
    P = {}
    beta = alpha / 2
    epsilon = alpha / (2 * phi)
    n = len(G.nodes())
    t = phi * pow(n * math.log(n), 0.5) / beta
    t = (n - 1 ) if t >= n else t
    V_dt = set(random.sample(list(G.nodes()), int(t)))
    R, W = INITIALIZATION(G, v, V_dt, l, epsilon)
    for i in range(k):
        R_v = DELETEEDGE(G, W, v, V_dt, P)
        arg_max = max(R_v)
        P[i] = arg_max
        G.remove_edges_from([arg_max])
    return P

In [79]:
# Example usage of the delete_edge function
G = nx.Graph()
nodes = [0, 1, 2, 3, 4]
edges = [(0, 2), (4, 2), (1, 2), (1, 3), (2, 3), (1, 4)]
G.add_nodes_from(nodes)
G.add_edges_from(edges)
k = 2
v = 0
l = 5
alpha = 0.05
phi = 5
P = FASTICM(G, k, v, l, alpha, phi)
print(f"Top-k edges need to remove {P}")

Top-k edges need to remove {0: (2, 4), 1: (2, 3)}
